## Implement Binary Search Tree

### Implement Map Data Structure

In [4]:
######################
# Map Data Structure #
#####################

### Construct Map Base Class for inherantace ###

from collections import MutableMapping

class MapBase(MutableMapping):
    """ Our own abstract base class that includes a nonpublic _Item class 
        for use in other map implementation.
    """
    
    # ------------- nested _Item class ------------ #
    class _Item:
        """ Lightweight composite to store key-value pairs as map items."""
        __slots__ = '_key', '_value'
        
        def __init__(self, k, v):
            self._key = k
            self._value = v
        
        def __eq__(self, other):
            """ compare items based on their keys"""
            return self._key == other._key
        
        def __ne__(self, other):
            """ opposite of __eq__"""
            return not(self == other)
        
        def __It__(self, other):
            """ compare items based on their keys"""
            return self._key < other._key
        

### Implement Tree Map

In [15]:
################################
# Implement Binary Search Tree
################################

from Tree import *

class TreeMap(LinkedBinaryTree, MapBase):
    """Sorted map implementation using a binary search tree."""
    
    # ------------- override Position class -------------- #
    class Position(LinkedBinaryTree.Position):
        
        def key(self):
            """Return key of map's key-value pair."""
            return self.element()._key
        
        def value(self):
            """Return value of map's key-value pair."""
            return self.element()._value
        
    # --------------- nonpublic utilities --------------- #
    def _subtree_search(self, p, k):
        """Return Position of p's subtree having key k, or last node searched."""
        
        if k == p.key():                           # found match
            return p
        elif k < p.key():                          # search left subtree
            if self.left(p) is not None:
                return self._subtree_search(self.left(p), k) 
        else:                                      # search right subtree
            if self.right(p) is not None:
                return self._subtree_search(self.right(p), k)
        return p                                   # unsuccessful search
    
    def _subtree_first_position(self, p):
        """Return Position of first item in subtree rooted at p."""
        walk = p
        while self.left(walk) is not None:         # keep walking left
            walk = self.left(walk)
        return walk
    
    def _subtree_last_position(self, p):
        """Return Position of last item in subtree rooted at p."""
        walk = p
        while self.right(walk) is not None:        # keep walking right
            walk = self.right(walk)
        return walk
    
    # ---------------- navigation methods --------------- #
    def first(self):
        """Return the first Position in the tree (or None if empty)."""
        return self._subtree_first_position(self.root()) if len(self) > 0 else None
    
    def last(self):
        """Return the last Position in the tree (or None if empty)."""
        return self._subtree_last_position(self.root()) if len(self) > 0 else None
    
    def before(self, p):
        """ Return the Position just before p in the natural order.
            Return None if p is the first position.
        """
        
        self._validate(p)    # inherited from LinkedBinaryTree
        if self.left(p): 
            # predesessor is the rightmost position of p's left subtree.
            return self._subtree_last_position(self.left(p))
        else:
            # predesessor is the nearest ancesor having p in its right subtree.
            walk = p
            above = self.parent(walk)
            # walk upward
            while above is not None and walk == self.left(above):
                walk = above
                above = self.parent(walk)
            return above
        
    def after(self, p):
        """ Return the Position just after p in the natural order.
            Return None if p is the last position.
        """
        
        self._validate(p)    # inherited from LinkedBinaryTree
        if self.right(p): 
            # successor is the leftmost position of p's right subtree.
            return self._subtree_first_position(self.right(p))
        else:
            # successor is the nearest ancesor having p in its left subtree.
            walk = p
            above = self.parent(walk)
            # walk upward
            while above is not None and walk == self.right(above):
                walk = above
                above = self.parent(walk)
            return above
    
    # ----------------- Operation Methods ----------------- #
    def find_position(self, k):
        """Return position with key k, or else neighbor (or None if empty)."""
        if self.is_empty():
            return None
        else:
            p = self._subtree_search(self.root(), k)
            self._rebalance_access(p)   # hook for balanced tree subclasses
            return p
    
    def find_min(self):
        """Return (key, value) pair with minimum key (or None if empty)."""
        if self.is_empty():
            return None
        else:
            p = self.first()
            return (p.key(), p.value())
        
    def find_ge(self, k):
        """ Return (key, value) pair with least key greater than or equal to k.
            Return None if there does not exist such a key.
        """
        if self.is_empty():
            return None
        else:
            p = self.find_position(k)     # may not find exact match
            if p.key() < k:               # p's key is too small
                p = self.after(p)
            return (p.key(), p.value()) if p is not None else None
    
    def find_range(self, start, stop):
        """ Iterate all (key, value) pairs such that start <= key < stop.
            If start is None, iteration begins with minimum key of map.
            If stop is None, iteration continues through the maximum key of map.
        """
        
        if not self.is_empty():
            if start is None:
                p = self.first()
            else:
                # initialize p with logic similar to find_ge
                p = self.find_position(start)
                if p.key() < start:
                    p = self.after(p)
            
            while p is not None and (stop is None or p.key() < stop):
                yield (p.key(), p.value())
                p = self.after(p)
    
    def __getitem__(self, k):
        """Return value associated with key k (raise KeyError if not found)."""
        if self.is_empty():
            raise KeyError('Key Error: ' + repr(k))
        else:
            p = self._subtree_search(self.root(), k)
            self._rebalance_access(p)     # hook for balanced tree subclassess
            if k != p.key():
                raise KeyError('Key Error: ' + repr(k))
            return p.value()
    
    def __setitem__(self, k, v):
        """Assign value v to key k, overwriting existing value if present."""
        if self.is_empty():
            leaf = self._add_root(self._Item(k,v))    
            # _add_root method is from LinkedBinaryTree & _Item class is from MapBase
        else:
            p = self._subtree_search(self.root(), k)
            if p.key() == k:
                p.element()._value = v     # replace existing item's value
                self._rebalance_access(p)  # hook for balanced tree subclasses
                return
            else:
                item = self._Item(k, v)
                if p.key() < k:
                    leaf = self._add_right(p, item)  # inherited from LinkedBinaryTree
                else:
                    leaf = self._add_left(p, item)   # inherited from LinkedBinaryTree
        
        self._rebalance_insert(leaf)                 # hook for balanced tree subclasses
        
    def __iter__(self):
        """Generate an iteration of all keys in the map in order."""
        p = self.first()
        while p is not None:
            yield p.key()
            p = self.after(p)
    
    # --- delete method for deleting an item either by position or by key --- #
    def delete(self, p):
        """Remove the item at given Position."""
        self._validate(p)        # inherited from LinkedBinaryTree
        
        # p has two children
        if self.left(p) and self.right(p):
            replacement = self._subtree_last_position(self.left(p))
            self._replace(p, replacement.element())
            p = replacement      # from LinkedBinaryTree
        
        # p has at most one child
        parent = self.parent(p)
        self._delete(p)          # inherited from LinkedBinaryTree
        self._rebalance_delete(parent) # if root deleted, parent is None
        
    def __delitem__(self, k):
        """Remove item associated with key k (raise KeyError if not found)."""
        if not self.is_empty():
            p = self._subtree_search(self.root(), k)
            if k == p.key():
                self.delete(p)         # rely on positional version
                return                 # successful deletion complete
            self._rebalance_access(p)  # hook for balanced tree subclasses
        raise KeyError('Key Error: ' + repr(k))
    
    # ----------- rebalancing hooks for subclasses to inherite ----------- #
    def _rebalance_insert(self, p): pass
    def _rebalance_delete(self, p): pass
    def _rebalance_access(self, p): pass
    
    # ---- nonpublic utilities for balanced search tree subclasses ---- #
    def _relink(self, parent, child, make_left_child):
        """Relink parent node with child node (allow child to be None)"""
        
        if make_life_child:             # make it a left child
            parent._left = child
        else:                           # make it a right child
            parent._right = child
        if child is not None:           # make child point to parent
            child._parent = parent
        
    def _rotate(self, p):
        """Rotate Position p above its parent."""
        x = p._node
        y = x._parent          # assume parent exists
        z = y._parent          # garandparent (possibly None)
        if z is None:
            self._root = x     # x becomes root
            x._parent = None
        else:
            self._relink(z, x, y == z._left)  # x becomes a direct child of z
        
        # rotate x and y, including transfer of middle subtree
        if x == y._left:
            self._relink(y, x._right, True)   # x._right becomes left child of y
            self._relink(x, y, False)         # y becomes right child of x
        else:
            self._relink(y, x._left, False)   # x._left becomes right child of y
            self._relink(x, y, True)          # y becmoes left child of x
            
    def _restructure(self, x):
        """Peform tirnode restructure of Position x with parent/grandparent."""
        
        y = self.parent(x)       # position of x's parent node
        z = self.parent(y)       # position of y's parent node
        if (x == self.right(y)) == (y == self.right(z)):  # match alignments
            # perfrom single rotation if the three nodes are align
            self._rotate(y)      # single rotation of y
            return y             # y is new subtree root
        else:                    # opposite alignments
            # perfrom double rotation if the three nodes are not align
            self._rotate(x)      # double rotation of x
            self._rotate(x)
            return x             # x is new subtree root
        

### Implement AVL Tree

In [ ]:
#####################
# Implement AVL Tree
#####################

class AVLTreeMap(TreeMap):
    """Sorted map implementation using an AVL tree."""
    
    # -------------- nested _Node class --------------- #
    class _Node(TreeMap._Node):
        """Node class for AVL maintains height value for balancing."""
        __slots__ = '_height'   # additional data member to store height
        
        def __init__(self, element, parent= None, left= None, right= None):
            super().__init__(element, parent, left, right)
            self._height = 0    # will be recomputed during balancing
        
        def left_height(self):
            return self._left._height if self._left is not None else 0
        
        def right_height(self):
            return self._right_height if self._right is not None else 0
    
    # --------- positional-based utility methods ---------- #
    def _recompute_height(self, p):
        p._node._height = 1 + max(p._node.left_height(), p._node.right_height())
        
    def _isbalance(self, p):
        return abs(p._node.left_height() - p._node.right_height()) <= 1
    
    def _tall_child(self, p, favorleft = False):  # parameter controls tiebreaker
        if p._node.left_height() + (1 if favorleft else 0) > p._node_right_height():
            return self.left(p)
        else:
            return self.right(p)
        
    def _tall_grandchild(self, p):
        child = self._tall_child(p)
        # if child is on left, favor left grandchild; else favor right grandchild
        alignment = (child == self.left(p))
        return self._tall_child(child, alignment)
    
    def _rebalance(self, p):
        while p is not None:
            old_height = p._node._height        # 0 if new node
            if not self._isbalanced(p):
                # perform trinode restructuring, setting p to resulting root,
                # and recompute new local heights after the restructuring
                p = self._restructure(self._tall_grandchild(p))
                self._recompute_height(self.left(p))
                self._recompute_height(self.right(p))
            self._recompute_height(p)           # adjust for recent changes
            if p._node._height == old._height:  # check if the height change
                p = None                        # no further changes needed
            else:
                p = self.parent(p)              # repeat with parent
                
    # ------------- override balancing hooks ------------- #
    def _rebalance_insert(self, p):
        self._rebalance(p)
    
    def _rebalance_delete(self, p):
        self._rebalance(p)


In [21]:
#####################################################
# HW 3: Implement a set using Binary Search Tree
# print out the rank of the given number
#####################################################

test_case = int(input())

for case in range(test_case):
    Tree = TreeMap()
    (N, Q) = [int(e) for e in input().split()]
    
    for i in range(Q):
        (act, n) = input().split()
        if act == 'add':
            Tree.__setitem__(n, n)      
            # in this simple case, key and value are set to the same
        else:
            rank = []
            for key in Tree.__iter__():
                rank.append(key)
            print(rank.index(n) + 1)    


1
10 8
add 6
add 8
add 9
rank 8
2
add 4
add 8
add 7
rank 9
5
